### Setup

In [20]:
import sqlite3
import pandas as pd
import numpy as np
import wget
import tarfile
from pathlib import Path

In [2]:
con = sqlite3.connect("GEOmetadb.sqlite")
cursor = con.cursor()
cursor.execute("select name from sqlite_master where type='table'")
print(cursor.fetchall())
# cur = con.cursor()

[('gse',), ('gpl',), ('gsm',), ('gse_gsm',), ('gse_gpl',), ('gds',), ('gds_subset',), ('sMatrix',), ('geodb_column_desc',), ('geoConvert',), ('metaInfo',), ('gse_ft',), ('gse_ft_data',), ('gse_ft_idx',), ('gse_ft_content',), ('gse_ft_docsize',), ('gse_ft_config',)]


In [5]:
# table = "gse"
# cols = "title,gse,summary,overall_design,supplementary_file"
# cursor.execute(f"create virtual table {table}_ft using fts5({cols})")
# cursor.execute(f"insert into {table}_ft select {cols} from {table}")

In [3]:
cols = "title,gse,summary,overall_design,supplementary_file"
gse_map = {i:ix for ix, i in enumerate(cols.split(","))}

### Query Process

In [26]:
get_wigs_from_ftps(get_genus_ftps("Campylobacter"))

In [25]:
def get_genus_ftps(genus):

    query = f"""SELECT * 
            FROM gse_ft WHERE gse_ft
            MATCH '{genus}'"""

    query = f"""SELECT *
            FROM gse_ft WHERE gse_ft
            MATCH 'RNA AND seq'
            AND gse_ft
            MATCH '{genus}'
            """
    results = cursor.execute(query).fetchall()

    positive_terms = "wild WT dRNA-".split()
    negative_terms = "plasmid".split()

    new_results = []
    for i in results:
        descriptive_text = i[gse_map["title"]] + i[gse_map["summary"]] + i[gse_map["overall_design"]]
        add = True
        for n in negative_terms:
            if n in descriptive_text:
                add = False
                break
        if add:
            new_results.append(i)

    results = new_results

    ftps = []

    for ix, i in enumerate(results):
        descriptive_text = i[gse_map["title"]] + i[gse_map["summary"]] + i[gse_map["overall_design"]]
        for p in positive_terms:
            if p in descriptive_text:
                if i[-1] is None:
                    continue
                if "RAW.tar" in i[-1]:
                    ftps.append([j for j in i[-1].split(";") if "RAW" in j][0].strip())
                break
    return ftps

def get_wigs_from_ftps(ftps, path="."):
    path = Path(path)
    for file in ftps:
        extracted = False
        wget.download(file)
        file = file.split("/")[-1]
        with tarfile.open(file, "r:") as tar:
            names = tar.getmembers()
            wig_files = [n for n in names if ".wig" in n.name] # and ("wt" in n.name or "WT" in n.name) ]
            if len(wig_files) > 0:
                tar.extractall(path=path, members=wig_files)
                
        os.remove(file)
        

In [24]:
cs = [i[0] for i in cursor.execute("select * from gsm").description]
r = cursor.execute("select * from gsm where gsm='GSM1331409'")
rs = list(r.fetchone())
for ix , i in enumerate(cs):
    print(f"{i}: {rs[ix]}")

ID: 1254998.0
title: LB 0.4 B1 TEX neg L1 GA
gsm: GSM1331409
series_id: GSE55199
gpl: GPL17024
status: Public on Oct 18 2014
submission_date: 2014-02-20
last_update_date: 2014-10-18
type: SRA
source_name_ch1: Escherichia coli MG1655 cells
organism_ch1: Escherichia coli str. K-12 substr. MG1655
characteristics_ch1: strain: MG1655;	medium: LB;	growth phase: exponential
molecule_ch1: total RNA
label_ch1: None
treatment_protocol_ch1: Samples were mixed, incubated on ice for 10 min after which cells were collected by centrifugation at 4150 rpm at 4˚C for 10 min.  Cell pellets were snap frozen in an ethanol/dry ice slurry and stored at -80°C until total RNA could be extracted.
extract_protocol_ch1: Hot phenol (Sharma et al. 2010, Bloomberg 1990, EMBO J);	The libraries were generated by Vertis Biotechnologie AG (Munich, Germany). The samples were poly(A)-tailed by using poly(A) polymerase. The 5'-PPP were removed using tobacco acid pyrophosphatase (TAP) followed by the ligation of the RNA ada

In [ ]:
"""
Process:

EDIT:
- Just need to do the GSE cause you can get the raw tar files of the gseeriment with 
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE1nnn/GSE1000/suppl/GSE1000_RAW.tar formatted
in the same way as below, then we can just parse through these

Probably prioritize those which have wig files

Search using fts over the GSE to get the project descriptions and then search for 
GSMs which have the same GSE in their "series_id" (or just skip straight to fts on 
the GSMs) and find those with the proper files/description in  "source_name_ch1,
organism_ch1, and treatment_protocol_ch1".

Then just wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM*nnn/GSM*/suppl/* 
to get all supp files, even if not listed
"""